# Step 1 - Scraping

In [1]:
# Import dependencies
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import time

### NASA Mars News
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
Assign the text to variables that you can reference later.

In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

nasa_url = 'https://mars.nasa.gov/news/'

browser.visit(nasa_url)

time.sleep(5)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

results = soup.find('ul', class_="item_list")

results_title = results.find('div', class_='content_title')

news_title = results_title.a.text

news_link = results.a['href']

news_p = results.find('div', class_='article_teaser_body').text

news_img = results.img['src']

print(f'Title: {news_title}')
print(f'Teaser: {news_link}')
print(f'URL: {news_p}')
print(f'Image link: {news_img}')
print('------')

Title: How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus 
Teaser: /news/8654/how-nasas-perseverance-mars-team-adjusted-to-work-in-the-time-of-coronavirus/
URL: Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.
Image link: /assets/overlay-arrow.png
------


### JPL Featured Mars Image
Use splinter to navigate the site and find the image url for the current Featured Mars Image. 
Assign the url string to a variable called featured_image_url.

In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(jpl_url)

time.sleep(5)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

featured_image_url = (f"https://www.jpl.nasa.gov{soup.find('a', class_='button fancybox')['data-fancybox-href']}")
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA23170_ip.jpg'

In [ ]:
# Closes browser window
browser.quit()

### Mars Weather Tweet
Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. 
Save the tweet text for the weather report as a variable called mars_weather.

In [4]:
request = requests.get('https://twitter.com/marswxreport?lang=en')

soup = BeautifulSoup(request.content, 'html.parser')

mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

mars_weather

'InSight sol 495 (2020-04-17) low -94.0ºC (-137.2ºF) high -4.2ºC (24.4ºF)\nwinds from the WNW at 4.6 m/s (10.4 mph) gusting to 16.7 m/s (37.3 mph)\npressure at 6.60 hPapic.twitter.com/SkjAdWsePB'

### Mars Facts
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

In [10]:
space_facts = 'https://space-facts.com/mars/'

# Use Pandas' `read_html` to parse the url
tables = pd.read_html(space_facts)

df = tables[0]

df.columns = ['Index', 'Value']

df.set_index('Index', inplace=True)

# df
fact_dict = df.to_dict()

fact_dict['Value']

# df.to_html('../Resources/mars_table.html')

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.38 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-87 to -5 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}

### Mars Hemispheres

Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

In [6]:
mars_pics_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

request = requests.get(mars_pics_url)

soup = BeautifulSoup(request.content, 'html.parser')

results = soup.find_all('a', class_="itemLink product-item")

img_list = []

for result in results:
    
    img = (f"https://astrogeology.usgs.gov{result['href']}")
                    
                    
    pic_req = requests.get(img)
    
    
    pic_soup = BeautifulSoup(pic_req.content, 'html.parser')
           
    page_name = (pic_soup.find('h2', class_='title')).text
           
    title = page_name.split(' ')[0]
    
    results = pic_soup.find('li')
    
    full_pic = results.find('a')['href']
           
    img_list.append({'title': title, 'image_url': full_pic})
               
img_list

[{'title': 'Cerberus',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

# Step 2 - MongoDB and Flask Application

In [9]:
recent_mars_info={}
recent_mars_info['Latest News'] = {'Title':news_title, 'Teaser': news_p, 'URL': news_link, 'Image': news_img}
recent_mars_info['JPL Featured Image'] = featured_image_url
recent_mars_info['Current Mars Weather'] = mars_weather
recent_mars_info['Hemisphere images'] = img_list

df_dict = df.to_dict()

recent_mars_info['Mars Facts'] = df_dict['Value']


In [10]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.mars_db
collection = db.mars_info

collection.insert_one(recent_mars_info)

listings = db.mars_info.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5e9ca19339f4f4769f07e81b'), 'Latest News': {'Title': "NASA's Curiosity Keeps Rolling As Team Operates Rover From Home", 'Teaser': 'The team has learned to meet new challenges as they work remotely on the Mars mission.', 'URL': '/news/8647/nasas-curiosity-keeps-rolling-as-team-operates-rover-from-home/', 'Image': '/assets/overlay-arrow.png'}, 'JPL Featured Image': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA23170_ip.jpg', 'Current Mars Weather': 'InSight sol 495 (2020-04-17) low -94.0ºC (-137.2ºF) high -4.2ºC (24.4ºF)\nwinds from the WNW at 4.6 m/s (10.4 mph) gusting to 16.7 m/s (37.3 mph)\npressure at 6.60 hPapic.twitter.com/SkjAdWsePB', 'Hemisphere images': [{'title': 'Cerberus', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli', 'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis', 'image_ur